# Predição de modelos por meio de SAVIME e PYSAVIME 

__O objetivo deste notebook é demonstrar o recurso de predição de execução de modelos, 
por meio do sistema SAVIME em conjunto com a biblioteca de acesso PySavime. Serão utilizados os modelos criados no notebook Tutorial - Parte 01.__

Para execução deste tutorial é necessário que esteja inicializado o servidor tensorflow na porta 8501, bem como em execução o sistema Savime, na porta 65000. As dependências necessárias são especificadas nos notebooks anteriores. 

1. Definição dos diretórios e informações de configuração:

In [1]:
import os
import sys

# Necessário mudar o diretório de trabalho para o diretório raiz
if not 'notebooks' in os.listdir('.'):
    current_dir = os.path.abspath(os.getcwd())
    parent_dir = os.path.dirname(current_dir)
    os.chdir(parent_dir)

# Informado o caminho da biblioteca do savime
py_savime_path =  '/home/anderson/Programacao/Savime/pysavime'
sys.path.insert(0, py_savime_path)

# Informado o caminho do arquivo de dados: um json contendo informações a respeito 
# da partição de x e y utilizada na parte 01.
data_file = 'saved_models_elastic_net/data.json'

# Configuração do host e porta em que o SAVIME está escutando
host = '127.0.0.1'
port = 65000

2. Inclusão dos pacotes necessários para a execução do notebook

In [2]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import json
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

# Savime imports
from savime.client import Client
import schema.define as savime_define
import misc.commands as savime_op
from util.converter import DataVariableBlockConverter
from util.data_variable import DataVariableBlockOps

from src.predictor_consumer import PredictionConsumer

3. Carga dos dados a serem utilizados

In [12]:
# Leitura do arquivo de dados .json de entrada
with open(data_file, 'r') as _in:
    data = json.load(_in)

# Leitura dos arrays X e Y
output_dir = data['output_dir']
x_fp = os.path.join(output_dir, data['x_file_name'])
y_fp = os.path.join(output_dir, data['y_file_name'])

x_array = np.load(x_fp)
y_array = np.load(y_fp)

print("Array X")
print(x_array)

print("Array Y")
print(y_array)

Array X
[[-0.99998923 -0.9999818 ]
 [-0.99997866 -0.99996557]
 [-0.99996234 -0.99994934]
 ...
 [ 0.99995965  0.99997022]
 [ 0.9999744   0.99997724]
 [ 0.99998446  0.99998447]]
Array Y
[-1.98935628 -1.8733491  -2.05083874 ...  6.03294597  5.93841993
  6.00459001]


In [ ]:
x_c_fp = os.path.join(output_dir, 'x_data')
y_c_fp = os.path.join(output_dir, 'y_data')

# Salvar os arrays numpy em arquivo sem os metadados do numpy.
x_array.astype('float64').tofile(x_c_fp)
y_array.astype('float64').tofile(y_c_fp)

4. Preparação dos comandos a serem executados no Savime

In [16]:
# Definição dos datasets a serem utilizados:
num_observations = 100000
num_features     = 2
y_num_columns    = 1 

x_dataset = savime_define.file_dataset('x', x_c_fp, 'double', length=num_features)
y_dataset = savime_define.file_dataset('y', y_c_fp, 'double')

# O comando gerado será:
print(x_dataset.create_query_str(), y_dataset.create_query_str(), sep='\n')

CREATE_DATASET("x:double:2", "@/home/anderson/Programacao/Savime/notebooks/savime-notebooks/saved_models_elastic_net/x_data");
CREATE_DATASET("y:double:1", "@/home/anderson/Programacao/Savime/notebooks/savime-notebooks/saved_models_elastic_net/y_data");


In [17]:
# Definição do Tar a ser empregado:

# Dimensão index
index = savime_define.implicit_tar_dimension('index', 'int32', 1, num_observations)
x = savime_define.tar_attribute('x', 'double', num_features)
y = savime_define.tar_attribute('y', 'double', y_num_columns)
tar = savime_define.tar('tutorial', [index], [x, y])

# O comando gerado será:
print(tar.create_query_str())

CREATE_TAR("tutorial", "*", "implicit, index, int32, 1, 100000, 1", "x, double: 2 | y, double: 1");


In [18]:
# Carregamento do SubTar:
# Aqui de fato os dados serão carregados no Tar.  

subtar_index = savime_define.ordered_subtar_dimension(index, 1, num_observations)
subtar_x = savime_define.subtar_attribute(x, x_dataset)
subtar_y = savime_define.subtar_attribute(y, y_dataset)
subtar = savime_define.subtar(tar, [subtar_index], [subtar_x, subtar_y])

print(subtar.load_query_str())

LOAD_SUBTAR("tutorial", "ordered, index, #1,#100000", "x, x | y, y")


Execução dos comandos no Savime

In [19]:
# 1. Conexão é aberta e fechada com o SAVIME (contexto with)
# 2. Criação de um objeto de execução de comandos vinculado à conexão criada.
# 3. a) Criação dos datasets
#    b) Criação do subtar
#    c) Carregamento dos datasets por meio de um subtar

with Client(host=host, port=port) as client:
    client.execute(savime_op.create(x_dataset))
    client.execute(savime_op.create(y_dataset))
    client.execute(savime_op.create(tar))
    client.execute(savime_op.load(subtar))

2020-02-14 17:36:06 [Client]:ERROR: Query handler response message: Error during operation execution: Dataset x already exists.
2020-02-14 17:36:06 [Client]:ERROR: Query handler response message: Error during operation execution: Dataset y already exists.
2020-02-14 17:36:07 [Client]:ERROR: Query handler response message: Error during operation execution: tutorial TAR already exists.
2020-02-14 17:36:07 [Client]:ERROR: Query handler response message: Error during operation execution: This new subtar definition intersects with already existing subtar!


5. Verificando se os comandos foram executados corretamente

In [20]:
# Os objetos abaixo são utilizados para converter do container mais genérico DataVariableBlock, retornado 
# como resposta da consulta, em  elmentos xarray e pandas. 
# Note que DataVariableBlocks contêm dois atributos: dims e attrs. Cada um desses contém um
# dicionário do tipo nome_array: array, onde array é um numpy array.

xarray_converter = DataVariableBlockConverter('xarray')
pandas_converter = DataVariableBlockConverter('pandas')

# Efetuar select no SAVIME a fim de verificar se o TAR for criado de forma adequada
with Client(host=host, port=port) as client:
    responses = client.execute(savime_op.select(tar))
        
# Em geral, o retorno do SAVIME a uma consulta é dado por subtar. Ou seja, se o tar contem n subtars então
# a variável responses acima será uma lista com n DataVariableBlocks. Abaixo, eles são concatenados.
data_variable_block = DataVariableBlockOps.concatenate(responses)

# E abaixo convertidos
xdataset_response = xarray_converter(data_variable_block)
pandas_response = pandas_converter(data_variable_block)

In [21]:
# Obs.: 
# 1) Como xarray representa arrays como matrizes multidimensionais densas e SAVIME é mais genérico, 
# abarcando matrizes esparsas, é preciso manter uma máscara _mask_ identificando se determinado
# elemento na matriz está presente ou não.
# 2) Dataframes pandas é intrinsicamente uma estrutura tabular. Para permitir a representação de atributos
# com uma segunda dimensão (matrizes) emprega-se índices múltiplos. Note a coluna x abaixo.
print('XDATASET')
print(xdataset_response)
print('DATAFRAME')
print(pandas_response)

XDATASET
<xarray.Dataset>
Dimensions:  (_0_: 2, index: 100000)
Coordinates:
  * index    (index) int32 1 2 3 4 5 6 ... 99995 99996 99997 99998 99999 100000
  * _0_      (_0_) int64 0 1
    _mask_   (index, _0_) bool True True True True True ... True True True True
Data variables:
    x        (index, _0_) float64 -1.0 -1.0 -1.0 -1.0 -1.0 ... 1.0 1.0 1.0 1.0
    y        (index) float64 -1.989 -1.873 -2.051 -1.94 ... 6.033 5.938 6.005
DATAFRAME
               x                   y
               0         1         0
index                               
1      -0.999989 -0.999982 -1.989356
2      -0.999979 -0.999966 -1.873349
3      -0.999962 -0.999949 -2.050839
4      -0.999921 -0.999907 -1.939803
5      -0.999897 -0.999871 -2.060704
...          ...       ...       ...
99996   0.999898  0.999903  5.969954
99997   0.999909  0.999938  5.953251
99998   0.999960  0.999970  6.032946
99999   0.999974  0.999977  5.938420
100000  0.999984  0.999984  6.004590

[100000 rows x 3 columns]


## 2.3 Instanciação TFX e execução de consultas preditivas

### Para executar os passos abaixo é nessário que o servidor de modelos esteja rodando.

Execute o comando abaixo:
`tensorflow_model_server --rest_api_port=8501 --model_config_file=ARQUIVO_DE_MODELOS` 
Note que você deve trocar ARQUIVO_DE_MODELOS pelo caminho do arquivo no qual os modelos foram registrados. Esse arquivo é o `models.config` dentro da pasta `saved_models`.